<a href="https://colab.research.google.com/github/Yihe0917/Rust-Fundamentals/blob/main/Quant_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install yfinance


In [4]:
import yfinance as yf
import pandas as pd
import numpy as np


Tasks 1. Extract historical price data for the stocks MSFT, AAPL, NVDA, AMZN, GOOG, META, and TSLA
using the Yahoo Finance API (yfinance). The data should span from 2013-01-01 to 2023-12-31

In [24]:
def get_stock_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    stock_prices = data["Close"]
    #print(data["Close"])  # Print the first few rows to check column names
    return stock_prices

tickers = ['MSFT', 'AAPL', 'NVDA', 'AMZN', 'GOOG', 'META', 'TSLA']
start_date = '2013-01-01'
end_date = '2023-12-31'

stock_data = get_stock_data(tickers, start_date, end_date)
print(stock_data)

[*********************100%***********************]  7 of 7 completed

Ticker            AAPL        AMZN        GOOG        META        MSFT  \
Date                                                                     
2013-01-02   16.669020   12.865500   17.949236   27.893450   22.362123   
2013-01-03   16.458609   12.924000   17.959660   27.664324   22.062557   
2013-01-04   16.000172   12.957500   18.314550   28.650557   21.649639   
2013-01-07   15.906051   13.423000   18.234638   29.308046   21.609169   
2013-01-08   15.948851   13.319000   18.198652   28.949413   21.495811   
...                ...         ...         ...         ...         ...   
2023-12-22  192.444580  153.419998  142.209045  352.045197  371.055695   
2023-12-26  191.897873  153.410004  142.308670  353.479675  371.134949   
2023-12-27  191.997269  153.339996  140.933624  356.468323  370.550507   
2023-12-28  192.424713  153.380005  140.774185  356.956482  371.749115   
2023-12-29  191.380966  151.940002  140.425446  352.613068  372.502014   

Ticker           NVDA        TSLA  
D

In [25]:
print(stock_data.index)  # Check available dates


DatetimeIndex(['2013-01-02', '2013-01-03', '2013-01-04', '2013-01-07',
               '2013-01-08', '2013-01-09', '2013-01-10', '2013-01-11',
               '2013-01-14', '2013-01-15',
               ...
               '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20',
               '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', name='Date', length=2768, freq=None)


Task 2. Implement the Double Bollinger Band indicator to generate buy and sell signals based on the
specified strategy

In [30]:
def calculate_bollinger_bands(data, period, std_dev_mult):
    rolling_mean = data.rolling(window=period).mean()
    rolling_std = data.rolling(window=period).std()

    upper_band = rolling_mean + (rolling_std * std_dev_mult)
    lower_band = rolling_mean - (rolling_std * std_dev_mult)

    return pd.DataFrame({'Upper': upper_band, 'Lower': lower_band, 'Moving Average': rolling_mean})

period=20
std_dev_mult=2

# Example usage for MSFT:
msft_data = stock_data['MSFT']
bollinger_bands = calculate_bollinger_bands(msft_data, period, std_dev_mult)
print(bollinger_bands.head())

            Upper  Lower  Moving Average
Date                                    
2013-01-02    NaN    NaN             NaN
2013-01-03    NaN    NaN             NaN
2013-01-04    NaN    NaN             NaN
2013-01-07    NaN    NaN             NaN
2013-01-08    NaN    NaN             NaN


In [32]:
def generate_trading_signals(data, upper_band, lower_band):
    signals = pd.Series(0, index=data.index)

    # Buy signal: Price crosses above the upper band
    signals[data > upper_band] = 1

    # Sell signal: Price crosses below the lower band
    signals[data < lower_band] = -1

    # To avoid consecutive signals, we only keep the signal when it changes
    signals = signals.replace(0, np.nan).ffill().fillna(0) #Forward fill
    signals = signals[signals.shift() != signals]


    return signals

# Example Usage
msft_signals = generate_trading_signals(msft_data, bollinger_bands['Upper'], bollinger_bands['Lower'])
print(msft_signals.head())


Date
2013-01-02    0.0
2013-03-28    1.0
2013-06-20   -1.0
2013-07-11    1.0
2013-07-19   -1.0
dtype: float64


Task 3. Design a backtesting loop to simulate trading with these signals. Assume an initial capital of
$10,000. The minimum transaction size is set at 1 share per trade

In [35]:
def backtest(data, signals, initial_capital, min_trade_size):
    positions = pd.Series(0, index=data.index)  # Number of shares held
    cash = initial_capital  # Start with initial capital
    portfolio_value = pd.Series(index=data.index, dtype='float64')  # Store portfolio value
    trades = []  # Store trade history

    for i, price in data.items():
        signal = signals.get(i, 0)  # Get signal, default to 0 if no signal for that date
        idx = data.index.get_loc(i)  # Get integer index of the timestamp

        # Get previous position (handle first row case)
        prev_position = positions.iloc[idx - 1] if idx > 0 else 0

        if signal == 1:  # Buy
            shares_to_buy = int(cash / price)  # Max shares we can buy
            shares_to_buy = max(shares_to_buy, min_trade_size)  # Ensure at least min_trade_size
            cost = shares_to_buy * price

            if cash >= cost:
                positions.loc[i] = prev_position + shares_to_buy  # Add shares
                cash -= cost
                trades.append({'date': i, 'action': 'buy', 'price': price, 'shares': shares_to_buy})
            else:
                positions.loc[i] = prev_position  # Not enough cash, hold

        elif signal == -1:  # Sell
            shares_to_sell = prev_position  # Sell all shares if we have any
            if shares_to_sell > 0:
                revenue = shares_to_sell * price
                cash += revenue
                positions.loc[i] = 0  # Sold all shares
                trades.append({'date': i, 'action': 'sell', 'price': price, 'shares': shares_to_sell})
            else:
                positions.loc[i] = prev_position  # No shares to sell, hold

        else:
            positions.loc[i] = prev_position  # Hold

        # Calculate portfolio value
        portfolio_value.loc[i] = cash + (positions.loc[i] * price)

    trades_df = pd.DataFrame(trades)
    return portfolio_value, trades_df


# Example Usage
initial_capital = 10000
min_trade_size = 1

msft_portfolio_value, msft_trades = backtest(msft_data, msft_signals, initial_capital, min_trade_size)
print(msft_portfolio_value.tail())
print(msft_trades)

Date
2023-12-22    36808.194460
2023-12-26    36816.040621
2023-12-27    36758.180849
2023-12-28    36876.843081
2023-12-29    36951.380098
dtype: float64
         date action       price  shares
0  2013-03-28    buy   23.355442     428
1  2013-06-20   sell   27.530876     428
2  2013-07-11    buy   29.339413     401
3  2013-07-19   sell   25.812773     401
4  2013-08-23    buy   28.767981     360
5  2014-01-08   sell   29.828627     360
6  2014-01-31    buy   31.563616     340
7  2014-10-10   sell   37.496094     340
8  2015-04-23    buy   37.405594     341
9  2015-06-08   sell   39.724850     341
10 2015-10-02    buy   39.846905     340
11 2016-01-07   sell   45.925499     340
12 2016-03-16    buy   48.188076     324
13 2016-04-22   sell   45.909443     324
14 2016-05-25    buy   46.534111     320
15 2016-06-27   sell   43.239594     320
16 2016-07-12    buy   47.507309     291
17 2016-09-09   sell   50.498577     291
18 2016-10-21    buy   53.598030     274
19 2017-08-10   sell   65

In [36]:
def calculate_performance_metrics(portfolio_values, risk_free_rate=0.02):

    returns = portfolio_values.pct_change().dropna()
    total_return = (portfolio_values[-1] - portfolio_values[0]) / portfolio_values[0]
    annual_return = returns.mean() * 252  # Assuming 252 trading days in a year
    annual_volatility = returns.std() * np.sqrt(252)
    sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility

    # Calculate downside deviation for Sortino Ratio
    downside_returns = returns[returns < 0]
    downside_deviation = downside_returns.std() * np.sqrt(252)
    sortino_ratio = (annual_return - risk_free_rate) / downside_deviation if downside_deviation > 0 else np.nan

    # Calculate maximum drawdown
    peak = portfolio_values.cummax()
    drawdown = (portfolio_values - peak) / peak
    max_drawdown = drawdown.min()

    return {
        'Total Return': total_return,
        'Annual Return': annual_return,
        'Annual Volatility': annual_volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Maximum Drawdown': max_drawdown
    }

risk_free_rate=0.02

# Example Usage
msft_metrics = calculate_performance_metrics(msft_portfolio_value,risk_free_rate)
print(msft_metrics)


{'Total Return': 2.6951380098342894, 'Annual Return': 0.1335420758265792, 'Annual Volatility': 0.17011003399723273, 'Sharpe Ratio': 0.6674625426765022, 'Sortino Ratio': 0.6842031797182992, 'Maximum Drawdown': -0.2592063256470247}


<ipython-input-36-a9ef3b3ab676>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  total_return = (portfolio_values[-1] - portfolio_values[0]) / portfolio_values[0]


In [44]:
import numpy as np
import pandas as pd

def calculate_performance_metrics(portfolio_values, risk_free_rate):
    """
    Calculates key performance metrics for a trading strategy.

    Args:
        portfolio_values (pd.Series): Time series of portfolio values.
        risk_free_rate (float): Annual risk-free rate (default 2%).

    Returns:
        dict: Performance metrics (total return, annual return, volatility, Sharpe ratio, etc.).
    """

    returns = portfolio_values.pct_change().dropna()  # Daily returns

    # Fix index issues by using .iloc[]
    initial_value = portfolio_values.iloc[0]
    final_value = portfolio_values.iloc[-1]

    # 1. **Total Return**
    total_return = (final_value - initial_value) / initial_value

    # 2. **Annualized Return (CAGR)**
    num_days = len(portfolio_values)  # Total days in dataset
    annualized_return = (final_value / initial_value) ** (252 / num_days) - 1

    # 3. **Annualized Volatility**
    annualized_volatility = returns.std() * np.sqrt(252)

    # 4. **Sharpe Ratio**
    excess_returns = returns - (risk_free_rate / 252)  # Adjust daily risk-free rate
    if annualized_volatility != 0:
        sharpe_ratio = excess_returns.mean() / annualized_volatility
    else:
        sharpe_ratio = 0  # Avoid division by zero

    # 5. **Sortino Ratio** (Uses downside deviation instead of total volatility)
    downside_returns = returns[returns < 0]  # Only negative returns
    downside_volatility = downside_returns.std() * np.sqrt(252)
    sortino_ratio = excess_returns.mean() / downside_volatility if downside_volatility != 0 else 0

    # 6. **Maximum Drawdown (MDD)**
    rolling_max = portfolio_values.cummax()  # Peak portfolio value at each point
    drawdown = (portfolio_values - rolling_max) / rolling_max  # % drop from peak
    max_drawdown = drawdown.min()  # Worst drawdown

    # 7. **Calmar Ratio** (Annual return / Maximum Drawdown)
    calmar_ratio = annualized_return / abs(max_drawdown) if max_drawdown != 0 else 0

    # Store metrics in dictionary
    metrics = {
        "total_return": total_return,
        "annualized_return": annualized_return,
        "annualized_volatility": annualized_volatility,
        "sharpe_ratio": sharpe_ratio,
        "sortino_ratio": sortino_ratio,
        "max_drawdown": max_drawdown,
        "calmar_ratio": calmar_ratio
    }

    return metrics

risk_free_rate = 0.02
# Example Usage
msft_metrics = calculate_performance_metrics(msft_portfolio_value,risk_free_rate)
print(msft_metrics)


{'total_return': 2.6951380098342894, 'annualized_return': 0.12636036039696852, 'annualized_volatility': 0.17011003399723273, 'sharpe_ratio': 0.002648660883636913, 'sortino_ratio': 0.002715091983009123, 'max_drawdown': -0.2592063256470247, 'calmar_ratio': 0.4874894934818846}


In [45]:
all_metrics = {}
all_portfolios = {}
all_trades = {}

period=20
std_dev_mult=2
initial_capital = 10000
min_trade_size = 1
risk_free_rate=0.02

for ticker in tickers:
    stock_data = get_stock_data([ticker], start_date, end_date)  # Fetch data for single ticker
    data = stock_data[ticker]
    bollinger_bands = calculate_bollinger_bands(data, period, std_dev_mult)
    signals = generate_trading_signals(data, bollinger_bands['Upper'], bollinger_bands['Lower'])
    portfolio_value, trades = backtest(data, signals,initial_capital, min_trade_size)
    metrics = calculate_performance_metrics(portfolio_value,risk_free_rate)

    all_metrics[ticker] = metrics
    all_portfolios[ticker] = portfolio_value
    all_trades[ticker] = trades

print("All Metrics:")
print(all_metrics)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


All Metrics:
{'MSFT': {'total_return': 2.6951380098342894, 'annualized_return': 0.12636036039696852, 'annualized_volatility': 0.17011003399723273, 'sharpe_ratio': 0.002648660883636913, 'sortino_ratio': 0.002715091983009123, 'max_drawdown': -0.2592063256470247, 'calmar_ratio': 0.4874894934818846}, 'AAPL': {'total_return': 3.9638798824310304, 'annualized_return': 0.15703871166776118, 'annualized_volatility': 0.1905314561528328, 'sharpe_ratio': 0.0030003825735348314, 'sortino_ratio': 0.0033507404491196924, 'max_drawdown': -0.3111397998190479, 'calmar_ratio': 0.5047207453340635}, 'NVDA': {'total_return': 11.629439273744822, 'annualized_return': 0.2597098201873578, 'annualized_volatility': 0.31954096245854013, 'sharpe_ratio': 0.003240462708325947, 'sortino_ratio': 0.0038643861366171673, 'max_drawdown': -0.5070490499730048, 'calmar_ratio': 0.5121986131345373}, 'AMZN': {'total_return': 2.8508629219055175, 'annualized_return': 0.1306012896390134, 'annualized_volatility': 0.22371291767743276, '

In [46]:
import pandas as pd

# Parameters
periods_to_test = range(10, 51, 5)  # Test periods from 10 to 50 in steps of 5
std_dev_mult = 2
initial_capital = 10000
min_trade_size = 1
risk_free_rate = 0.02

best_params = {}  # Store best period per stock
best_metrics = {}  # Store best performance per stock
best_portfolios = {}  # Store best portfolio per stock
best_trades = {}  # Store best trade history per stock

for ticker in tickers:
    stock_data = get_stock_data([ticker], start_date, end_date)  # Fetch data for single ticker
    data = stock_data[ticker]

    best_sharpe = float('-inf')  # Initialize worst Sharpe Ratio

    for period in periods_to_test:
        # Calculate Bollinger Bands and generate trading signals
        bollinger_bands = calculate_bollinger_bands(data, period, std_dev_mult)
        signals = generate_trading_signals(data, bollinger_bands['Upper'], bollinger_bands['Lower'])

        # Run backtest
        portfolio_value, trades = backtest(data, signals, initial_capital, min_trade_size)

        # Evaluate performance
        metrics = calculate_performance_metrics(portfolio_value, risk_free_rate)

        # Select best-performing period (e.g., highest Sharpe Ratio)
        if metrics['sharpe_ratio'] > best_sharpe:
            best_sharpe = metrics['sharpe_ratio']
            best_params[ticker] = period
            best_metrics[ticker] = metrics
            best_portfolios[ticker] = portfolio_value
            best_trades[ticker] = trades

# Display results
print("Best Parameters for Each Stock:")
print(best_params)

print("\nBest Performance Metrics:")
print(best_metrics)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Best Parameters for Each Stock:
{'MSFT': 35, 'AAPL': 10, 'NVDA': 50, 'AMZN': 30, 'GOOG': 30, 'META': 40, 'TSLA': 10}

Best Performance Metrics:
{'MSFT': {'total_return': 3.338484093475342, 'annualized_return': 0.14294030244116218, 'annualized_volatility': 0.18619106172730626, 'sharpe_ratio': 0.0027929603189276796, 'sortino_ratio': 0.003084233356597845, 'max_drawdown': -0.37996173945914885, 'calmar_ratio': 0.37619656822454944}, 'AAPL': {'total_return': 4.786296056079864, 'annualized_return': 0.17330064058699834, 'annualized_volatility': 0.1655652824790898, 'sharpe_ratio': 0.0036828311747373415, 'sortino_ratio': 0.003661577184828153, 'max_drawdown': -0.25382595775956635, 'calmar_ratio': 0.6827538133477875}, 'NVDA': {'total_return': 24.874525656989217, 'annualized_return': 0.3447097579376015, 'annualized_volatility': 0.3456964657953827, 'sharpe_ratio': 0.0038444338627598054, 'sortino_ratio': 0.005115875051469757, 'max_drawdown': -0.5203153877388021, 'calmar_ratio': 0.6625015635913607}, 'A